In [16]:
import os
import plotly.express as px

from rasterio.enums import Resampling

import matplotlib.pyplot as plt

import basd
from matplotlib import pyplot
import numpy as np
import pandas as pd
import rioxarray as rio
import seaborn as sns
import xarray as xr

#### Reading NetCDF file
Reading in observational precipitation data from 1979-2014 supplied in the data directory.

In [17]:
pr_obs_hist = xr.open_dataset('../basd/data/pr_obs-hist_fine_1979-2014.nc')
pr_sim_hist = xr.open_dataset('../basd/data/pr_sim-hist_coarse_1979-2014.nc')
pr_sim_fut = xr.open_dataset('../basd/data/pr_sim-fut_coarse_2065-2100.nc')

### Running Bias Adjustment
#### Initializing our first `Adjustment` object
Here we are providing the three required data arrays and a variable name 'pr' representing the daily mean precipitation variable. Precipitation is assumed to follow a gamma distribution, and thus requires a lower bound and lower threshold parameter. We also set trend preservation mode to be 'mixed'. Thus, we set the parameter object directly specifying:

* Lower bound = 0
* Lower threshold = 0.0000011574
* Trend preservation = mixed
* Distribution = gamma

Otherwise, we are keeping the default parameter values for the bias adjustment.

In [18]:
params = basd.Parameters(lower_bound=0, lower_threshold=0.0000011574,
                         trend_preservation='mixed',
                         distribution='gamma',
                         if_all_invalid_use=0, n_iterations=10)
ba = basd.Adjustment(pr_obs_hist, pr_sim_hist, pr_sim_fut,
                     'pr', params, remap_grid=True )

#### Running Bias Adjustment
Here we run the bias adjustment for just one grid cell. We pass in a tuple with the indexes which we wish to adjust, (0,0), which in this case correspond to the coordinates 53.5 N, 12.5 E.

In [19]:
loc = dict(lat=1, lon=1)
sim_fut_ba_loc = ba.adjust_bias_one_location(loc)

In [20]:
pr_sim_fut_ba = ba.adjust_bias(n_jobs=-1)
#ba.save_adjustment_nc('../../../Documents/pr_sim_fut_ba.nc')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    7.3s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    7.4s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    7.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    6.9s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    7.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    7.0s finished


# Statistical Downscaling
We're now going to start to look at downscaling simulated data to observational data resolution using the statistical downscaling algorithm.

In [21]:
sd = basd.Downscaler(pr_obs_hist, pr_sim_fut_ba, 'pr', params)

In [22]:
loc = dict(lat=0, lon=0)
out = sd.downscale_one_location(loc)

Initial Coarse Value: 1.6547544419154292e-06
Initial Agg fine value: 2.9591480383146075e-05
Initial Coarse Value Scaled: 1.6547544419154292e-06
Agg fine value at iteration 0: 1.6547544419154313e-06
Agg fine value at iteration 1: 1.6547544419154427e-06
Agg fine value at iteration 2: 1.654754441915444e-06
Agg fine value at iteration 3: 1.6547544419154447e-06
Agg fine value at iteration 4: 1.6547544419154444e-06
Agg fine value at iteration 5: 1.6547544419154444e-06
Agg fine value at iteration 6: 1.654754441915445e-06
Agg fine value at iteration 7: 1.6547544419154453e-06
Agg fine value at iteration 8: 1.654754441915445e-06
Agg fine value at iteration 9: 1.6547544419154449e-06
Agg fine value at iteration 10: 1.6547544419154444e-06
Agg fine value at iteration 11: 1.6547544419154438e-06
Initial Coarse Value: 2.4458965981466463e-06
Initial Agg fine value: 3.0316635529596257e-05
Initial Coarse Value Scaled: 2.4458965981466463e-06
Agg fine value at iteration 0: 2.4458965981466446e-06
Agg fine va

In [32]:
init = pr_sim_fut_ba['pr'][loc].values
init_obs = sum(sum(pr_sim_fut_ba['pr'][dict(lat=[0,1], lon=[0,1])].values))
sum_weights_loc = sd.sum_weights[[0, 1]].repeat(2)
agg_out = np.dot(out.reshape((4, out.shape[2])).T, sum_weights_loc)
np.column_stack((init, agg_out, init_obs))

array([[1.65475444e-06, 9.51840624e-07, 6.57959845e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [1.76596325e-06, 9.07408032e-07, 1.76596325e-06],
       [2.07247695e-06, 1.08303563e-06, 3.29904151e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [31]:
sd.downscale()

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Initial Coarse Value: 1.6547544419154292e-06
Initial Agg fine value: 2.9591480383146075e-05
Initial Coarse Value Scaled: 1.6547544419154292e-06
Agg fine value at iteration 0: 1.6547544419154313e-06
Agg fine value at iteration 1: 1.6547544419154427e-06
Agg fine value at iteration 2: 1.654754441915444e-06
Agg fine value at iteration 3: 1.6547544419154447e-06
Agg fine value at iteration 4: 1.6547544419154444e-06
Agg fine value at iteration 5: 1.6547544419154444e-06
Agg fine value at iteration 6: 1.654754441915445e-06
Agg fine value at iteration 7: 1.6547544419154453e-06
Agg fine value at iteration 8: 1.654754441915445e-06
Agg fine value at iteration 9: 1.6547544419154449e-06
Agg fine value at iteration 10: 1.6547544419154444e-06
Agg fine value at iteration 11: 1.6547544419154438e-06
Initial Coarse Value: 2.4458965981466463e-06
Initial Agg fine value: 3.0316635529596257e-05
Initial Coarse Value Scaled: 2.4458965981466463e-06
Agg fine value at iteration 0: 2.4458965981466446e-06
Agg fine va

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s


Initial Coarse Value: 2.6177212930633686e-06
Initial Agg fine value: 2.089652154405688e-05
Initial Coarse Value Scaled: 2.6177212930633686e-06
Agg fine value at iteration 0: 2.617721293063364e-06
Agg fine value at iteration 1: 2.6177212930633627e-06
Agg fine value at iteration 2: 2.6177212930633627e-06
Agg fine value at iteration 3: 2.617721293063364e-06
Agg fine value at iteration 4: 2.6177212930633653e-06
Agg fine value at iteration 5: 2.6177212930633653e-06
Agg fine value at iteration 6: 2.617721293063367e-06
Agg fine value at iteration 7: 2.6177212930633665e-06
Agg fine value at iteration 8: 2.617721293063367e-06
Agg fine value at iteration 9: 2.617721293063367e-06
Agg fine value at iteration 10: 2.617721293063367e-06
Agg fine value at iteration 11: 2.6177212930633686e-06
Initial Coarse Value: 2.6441300633450737e-06
Initial Agg fine value: 2.261017959472437e-05
Initial Coarse Value Scaled: 2.6441300633450737e-06
Agg fine value at iteration 0: 2.644130063345074e-06
Agg fine value at

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.4s finished


<xarray.Dataset>
Dimensions:      (lon: 4, lat: 4, time: 13148)
Coordinates:
  * lon          (lon) float64 12.25 12.75 13.25 13.75
  * lat          (lat) float64 53.75 53.25 52.75 52.25
  * time         (time) datetime64[ns] 2065-01-01 2065-01-02 ... 2100-12-31
    spatial_ref  int32 0
Data variables:
    pr           (time, lat, lon) float32 1.655e-06 1.241e-06 ... 0.0 0.0